# Reversible Deep Equilibrium Models (RevDEQ) - Colab

このノートブックはGoogle ColabでRevDEQを実行するためのものです。


In [ ]:
# 必要なライブラリのインストール
%pip install -q torch transformers datasets accelerate tqdm wandb pyyaml tensorboard

# GPU確認
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")


In [ ]:
# リポジトリのクローン（GitHubに公開後）
# !git clone https://github.com/yourusername/RevDEQ.git
# %cd RevDEQ

# または、必要なファイルを直接アップロードする場合
# このセルを実行して、revdeq/フォルダとtrain.pyをアップロードしてください
import os
print("Current directory:", os.getcwd())


In [ ]:
# モジュールのインポート
import sys
if '/content/RevDEQ' in os.listdir('/content'):
    sys.path.append('/content/RevDEQ')
else:
    sys.path.append('.')

from revdeq import RevDEQ, RevDEQConfig
from train import prepare_dataset, RevDEQDataset
from transformers import AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling


In [ ]:
# モデル設定
config = RevDEQConfig(
    hidden_size=768,
    num_layers=12,
    num_heads=12,
    intermediate_size=3072,
    dropout=0.1,
    max_position_embeddings=512,
    num_fixed_point_iterations=10,
    fixed_point_tol=1e-5,
    use_reversible=True,
)

# トークナイザーの読み込み
tokenizer = AutoTokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

config.vocab_size = len(tokenizer)

# モデルの初期化
model = RevDEQ(config)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")


In [ ]:
# データセットの準備
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
texts = [example["text"] for example in dataset if len(example.get("text", "").strip()) > 0]

train_dataset = RevDEQDataset(texts[:1000], tokenizer, max_length=512)  # サンプル用に1000件のみ
print(f"Dataset size: {len(train_dataset)}")


In [ ]:
# 学習設定
training_args = TrainingArguments(
    output_dir="./checkpoints",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=100,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    report_to="tensorboard",
    remove_unused_columns=False,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# トレーナーの初期化
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

# 学習開始
print("Starting training...")
trainer.train()

# モデルの保存
trainer.save_model()
print("Training completed!")


In [ ]:
# 推論のテスト
model.eval()
input_text = "The quick brown fox"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

with torch.no_grad():
    generated = model.generate(
        input_ids,
        max_length=50,
        temperature=1.0,
        top_k=50,
        top_p=0.9
    )

generated_text = tokenizer.decode(generated[0], skip_special_tokens=True)
print(f"Generated: {generated_text}")


## TensorBoardで学習進捗を確認

以下のコマンドでTensorBoardを起動できます（Colab内で実行する場合）:


In [ ]:
# TensorBoardの起動（Colabで使用する場合）
# %load_ext tensorboard
# %tensorboard --logdir ./checkpoints/logs
